# 1.import Module

In [2]:
import torch
from torchvision import datasets,transforms, models
from torch import nn,optim
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
import torchvision
import pandas as pd

# 2.Unzip file we need(train,valid,test)

In [ ]:
!unzip "/content/drive/My Drive/공모전/Validset.zip" -d "/content/drive/My Drive/공모전/valid"

Archive:  /content/drive/My Drive/공모전/Validset.zip
   creating: /content/drive/My Drive/공모전/valid/0/
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1001.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1004.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1006.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1017.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1022.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1026.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1027.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1033.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1035.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1049.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1054.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_1059.jpg  
  inflating: /content/drive/My Drive/공모전/valid/0/3_5_106

# 3.Prepare things that we need

In [3]:
train_transform = transforms.Compose([transforms.RandomRotation(30),
                                     transforms.RandomResizedCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])

valid_transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])

# Transform image to tensor and crop

In [4]:
trainset = datasets.ImageFolder(root="/content/drive/My Drive/공모전/train",transform=train_transform)
validset = datasets.ImageFolder(root="/content/drive/My Drive/공모전/valid",transform=valid_transform)

# Load Dataset and transform
# Trainset : 12000 images
# Validset : 4000 images

In [5]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size=100,shuffle=True) # Trainset is 12000 images so length of trainloader should be 120
validloader = torch.utils.data.DataLoader(validset,batch_size=64,shuffle=True) # As same as above, 4000 images in validset, 4000/64 = 62.5. As a result, length of validloader should be 63

#Load image by batch_size

In [6]:
model = models.vgg11(pretrained=True)
model

#Prepare pretrained model

Downloading: "https://download.pytorch.org/models/vgg11-bbd30ac9.pth" to /root/.cache/torch/hub/checkpoints/vgg11-bbd30ac9.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
for param in model.parameters():
    
    param.requires_grad = False
model.classifier = nn.Sequential(nn.Linear(25088,1024),
                                nn.ReLU(),
                                nn.Dropout(p=0.1),
                                nn.Linear(1024,20),
                                nn.LogSoftmax(dim=1))

#Modify classifier to match output layer

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(),lr=0.003)

#Set loss function and optimizer

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#Send model to GPU

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

# 4.Train Model

In [ ]:
epoch = 10
steps = 0
print_every = 10
train_losses, valid_losses = [],[]

for e in range(epoch):
    
    
    for images,labels in trainloader: # This trainloader for loop works until all 14000 images processed. As a result, when steps become 140, next epoch will be started.
        
        images, labels = images.to(device),labels.to(device)
        running_loss = 0
        steps += 1 # Set steps to save time. Because we shuffle images in trainloader, we don't have to use whole 14000 images. 
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps,labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        print("Train Steps {}".format(steps))

        # Train 10 times with batch size 100 in trainloader
        

        
        
        
        if steps % print_every == 0: # If steps is 10, 20, 30, 40... enter the valid session so that we can calculate valid loss and accuracy.
            valid_loss = 0
            accuracy = 0
            valid_step = 0
            
            with torch.no_grad():
                
                model.eval()
                
                for images, labels in validloader: 

                  valid_step += 1 # Define valid_step

                  images, labels = images.to(device),labels.to(device)
                    
                  log_ps = model(images)
                  valid_loss += criterion(log_ps,labels)
                  ps = torch.exp(log_ps)
                    
                  top_p, top_class = ps.topk(1,dim=1)
                    
                  equal = top_class == labels.view(*top_class.shape)
                    
                  accuracy += torch.mean(equal.type(torch.FloatTensor))

                  if valid_step == 5: # For saving time, we do validation 5 times with batch size 64. When we don't use this, because batch size of validloader is 64, 4000/64 = 62.5. As a result, we should continue to do 63 times.
                    break

                  
                
                model.train()
            
            train_losses.append(running_loss/10) # Because we just train 10 times with batch size 100, loss of train will be accumulated 10 times with batch size 100. For make average, we should devide it by 10.
            valid_losses.append(valid_loss/5) # As same as above, we do validation 5 times, we should devide valid_loss by 5 to calculate mean.

            print("------------------------------------------------\n",
                  "Epoch : {} / {}".format(e+1,epoch),
                  "Training_loss : {:.3f}".format(train_losses[-1]),
                  "Valid_loss : {:.3f}".format(valid_losses[-1]),
                  "Accuracy : {:.3f}".format(accuracy/5)) # As same as valid_loss, torch.mean(equal...) will be accumulated 5 times. So for make average, we should devide it by 5. 
            
            if (accuracy/5) >=0.95:

              sys.exit() # Stop training when accuracy exceed 0.95



         
                 

Train Steps 1
Train Steps 2
Train Steps 3
Train Steps 4
Train Steps 5
Train Steps 6
Train Steps 7
Train Steps 8
Train Steps 9
Train Steps 10
------------------------------------------------
 Epoch : 1 / 10 Training_loss : 0.032 Valid_loss : 0.262 Accuracy : 0.941
Train Steps 11
Train Steps 12
Train Steps 13
Train Steps 14
Train Steps 15
Train Steps 16
Train Steps 17
Train Steps 18
Train Steps 19
Train Steps 20
------------------------------------------------
 Epoch : 1 / 10 Training_loss : 0.043 Valid_loss : 0.284 Accuracy : 0.913
Train Steps 21
Train Steps 22
Train Steps 23
Train Steps 24
Train Steps 25
Train Steps 26
Train Steps 27
Train Steps 28
Train Steps 29
Train Steps 30
------------------------------------------------
 Epoch : 1 / 10 Training_loss : 0.041 Valid_loss : 0.474 Accuracy : 0.897
Train Steps 31
Train Steps 32
Train Steps 33
Train Steps 34
Train Steps 35
Train Steps 36
Train Steps 37
Train Steps 38
Train Steps 39
Train Steps 40
----------------------------------------

NameError: ignored

In [9]:
class_to_idx = trainset.class_to_idx
class_to_idx

# Check class that we should predict

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '12': 4,
 '13': 5,
 '14': 6,
 '15': 7,
 '16': 8,
 '17': 9,
 '18': 10,
 '19': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19}

# 5.Save model

In [ ]:
torch.save(model,"/content/drive/My Drive/공모전/checkpoint2.pth")

In [10]:
model_load = torch.load("/content/drive/My Drive/공모전/checkpoint2.pth")
model_load.class_to_idx = class_to_idx # Define class to model that we load

In [ ]:
model_load

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

# 6.Predict result

In [11]:
from PIL import Image
import numpy as np

def process_image(image):

    
    pil_im = Image.open(f'{image}' + '.jpg') # Load image from folder

    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                         [0.229, 0.224, 0.225])]) 
    
    pil_tfd = transform(pil_im) # Transform image to tensor
    
    array_im_tfd = np.array(pil_tfd)
    
    return pil_tfd # Return image as tensor

In [18]:
def predict(path,model):

  img = process_image(path) # Preprocess image
  img_add_dim = img.unsqueeze_(0) # Expand dimension to match with model input size
  img_device = img_add_dim.to(device) 

  model.cuda()

  model.eval()

  with torch.no_grad():
    output = model.forward(img_device)
    probs = torch.exp(output)

    top_p,top_class = probs.topk(1,dim=1)

    top_class_np = top_class.cpu().numpy().tolist()[0][0] # Because top_class type is cuda tensor, we should change this to cpu(). After than we can change it to numpy and list

    for i,v in class_to_idx.items(): # Mapping class for prediction

      if top_class_np == v:

        return int(i)

In [21]:
result = []

for i in range(0,3997):

  n1 = predict("/content/drive/My Drive/공모전/test/{}".format(i),model_load)
  print("Processing... {}".format(i))
  result.append(n1)

Processing... 0
Processing... 1
Processing... 2
Processing... 3
Processing... 4
Processing... 5
Processing... 6
Processing... 7
Processing... 8
Processing... 9
Processing... 10
Processing... 11
Processing... 12
Processing... 13
Processing... 14
Processing... 15
Processing... 16
Processing... 17
Processing... 18
Processing... 19
Processing... 20
Processing... 21
Processing... 22
Processing... 23
Processing... 24
Processing... 25
Processing... 26
Processing... 27
Processing... 28
Processing... 29
Processing... 30
Processing... 31
Processing... 32
Processing... 33
Processing... 34
Processing... 35
Processing... 36
Processing... 37
Processing... 38
Processing... 39
Processing... 40
Processing... 41
Processing... 42
Processing... 43
Processing... 44
Processing... 45
Processing... 46
Processing... 47
Processing... 48
Processing... 49
Processing... 50
Processing... 51
Processing... 52
Processing... 53
Processing... 54
Processing... 55
Processing... 56
Processing... 57
Processing... 58
Process

# 7.Make result dateframe

In [22]:
result_df = pd.DataFrame(result,columns=["combined_label"])

In [23]:
dic_plant = {0:3,
             1:3,
             2:4,
             3:4,
             4:4,
             5:5,
             6:7,
             7:7,
             8:8,
             9:8,
             10:10,
             11:11,
             12:13,
             13:13,
             14:13,
             15:13,
             16:13,
             17:13,
             18:13,
             19:13
             }

dic_disease = {
    0:5,
    1:20,
    2:2,
    3:7,
    4:11,
    5:8,
    6:1,
    7:20,
    8:6,
    9:9,
    10:20,
    11:14,
    12:1,
    13:6,
    14:9,
    15:15,
    16:16,
    17:17,
    18:18,
    19:20

}

# For mapping result, we use dictionary and define one by one

In [24]:
cl1 = result_df.combined_label # For mapping the result, we load original combined label

In [25]:
plant = []
disease = []

for i in range(0,3997):

  plant.append(dic_plant[int(cl1[i])]) # Mapping combined label to plant label
  disease.append(dic_disease[int(cl1[i])]) # Mapping combined label to disease label

In [26]:
plant_df = pd.DataFrame(plant)
disease_df = pd.DataFrame(disease)

In [27]:
file_name_list = []

for i in range(0,3997):

  file_name_list.append("{}.jpg".format(i))


file_name = pd.DataFrame(file_name_list,columns=["file_name"]) # Make 0~3996.jpg index for submission

In [28]:
result_df2 = pd.concat([file_name,plant_df,disease_df],axis=1)

In [29]:
result_df2

,file_name,0,0
0,0.jpg,3,5
1,1.jpg,3,20
2,2.jpg,4,2
3,3.jpg,4,7
4,4.jpg,4,11
...,...,...,...
3992,3992.jpg,13,15
3993,3993.jpg,13,16
3994,3994.jpg,13,17
3995,3995.jpg,13,18


In [30]:
result_df2.columns = ["file_name","Plant","Disease"]

In [31]:
result_df2.to_csv("result.tsv",index=False,header=False,sep="\t")